In [1]:
from IPython.display import Image
from sklearn import ensemble
from sklearn import tree
import pydotplus
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from math import sqrt
from scipy.stats import ttest_ind
from scipy.stats import ks_2samp
from sklearn.model_selection import cross_val_score
import timeit


df = pd.read_csv('C:\\Users\\Square Bear\\Documents\\Thinkful\\Datasets\\music.csv')

In [2]:
pd.options.display.max_columns = None
df.head()

,artist.hotttnesss,artist.id,artist.name,artist_mbtags,artist_mbtags_count,bars_confidence,bars_start,beats_confidence,beats_start,duration,end_of_fade_in,familiarity,key,key_confidence,latitude,location,longitude,loudness,mode,mode_confidence,release.id,release.name,similar,song.hotttnesss,song.id,start_of_fade_out,tatums_confidence,tatums_start,tempo,terms,terms_freq,time_signature,time_signature_confidence,title,year
0,0.401998,ARD7TVE1187B99BFB1,Casual,NaN,0.0,0.643,0.58521,0.834,0.58521,218.93179,0.247,0.581794,1.0,0.736,37.157357,California - LA,-63.933358,-11.197,0,0.636,300848,Fear Itself,ARV4KO21187FB38008,0.602120,SOMZWCG12A8C13C480,218.932,0.779,0.28519,92.198,hip hop,1.000000,4.0,0.778,I Didn't Mean To,0
1,0.417500,ARMJAGH1187FB546F3,The Box Tops,classic pop and rock,1.0,0.007,0.71054,1.000,0.20627,148.03546,0.148,0.630630,6.0,0.169,35.149680,"Memphis, TN",-90.048920,-9.843,0,0.430,300822,Dimensions,ARSZWK21187B9B26D7,NaN,SOCIWDW12A8C13D406,137.915,0.969,0.20627,121.274,blue-eyed soul,1.000000,4.0,0.384,Soul Deep,1969
2,0.343428,ARKRRTF1187B9984DA,Sonora Santanera,NaN,0.0,0.980,0.73152,0.980,0.73152,177.47546,0.282,0.487357,8.0,0.643,37.157357,Not available,-63.933358,-9.689,1,0.565,514953,Las Numero 1 De La Sonora Santanera,ARFSJUG11C8A421AAD,NaN,SOXVLOJ12AB0189215,172.304,0.482,0.42132,100.070,salsa,1.000000,1.0,0.000,Amor De Cabaret,0
3,0.454231,AR7G5I41187FB4CE6C,Adam Ant,uk,1.0,0.017,1.30621,0.809,0.81002,233.40363,0.000,0.630382,0.0,0.751,37.157357,"London, England",-63.933358,-9.013,1,0.749,287650,Friend Or Foe,AR4R0741187FB39AF2,NaN,SONHOTT12A8C13493C,217.124,0.601,0.56254,119.293,pop rock,0.988584,4.0,0.000,Something Girls,1982
4,0.401724,ARXR32B1187FB57099,Gob,NaN,0.0,0.175,1.06368,0.883,0.13576,209.60608,0.066,0.651046,2.0,0.092,37.157357,Not available,-63.933358,-4.501,1,0.371,611336,Muertos Vivos,ARUA62A1187B99D9B0,0.604501,SOFSOCN12A8C143F5D,198.699,1.000,0.13576,129.738,pop punk,0.887288,4.0,0.562,Face the Ashes,2007


In [3]:
df = df.drop(['artist_mbtags'], axis=1)

In [4]:
# this will take care of all the int or float columns
df = df.fillna(df.mean()) 

# This takes care of the Nan in 'artist_mbtags'
df['similar'] = df['similar'].fillna(df['similar'].mode()[0])
df['terms'] = df['terms'].fillna(df['terms'].mode()[0])
df['title'] = df['title'].fillna(df['title'].mode()[0])



In [5]:
df.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 34 columns):
artist.hotttnesss            10000 non-null float64
artist.id                    10000 non-null object
artist.name                  10000 non-null object
artist_mbtags_count          10000 non-null float64
bars_confidence              10000 non-null float64
bars_start                   10000 non-null float64
beats_confidence             10000 non-null float64
beats_start                  10000 non-null float64
duration                     10000 non-null float64
end_of_fade_in               10000 non-null float64
familiarity                  10000 non-null float64
key                          10000 non-null float64
key_confidence               10000 non-null float64
latitude                     10000 non-null float64
location                     10000 non-null object
longitude                    10000 non-null float64
loudness                     10000 non-null float64
mode       

In [6]:

df['good'] = df['song.hotttnesss'] >= 0.5
#df['bad'] = df['song.hotttnesss'] < 0.5

'''
for i in (df['song.hotttnesss']):
    if df['song.hotttnesss'] >= 0.5:
        df['good'] = 1
    elif df['song.hotttnesss'] < 0.5:
        df['bad'] = 0
    else:
        pass
'''

df['good'].astype(int)

music = df[['good','beats_confidence', 'duration', 'key', 'loudness', 'tempo']]
music.head()


,good,beats_confidence,duration,key,loudness,tempo
0,True,0.834,218.93179,1.0,-11.197,92.198
1,False,1.000,148.03546,6.0,-9.843,121.274
2,False,0.980,177.47546,8.0,-9.689,100.070
3,False,0.809,233.40363,0.0,-9.013,119.293
4,True,0.883,209.60608,2.0,-4.501,129.738


In [7]:
start_time = timeit.default_timer()

for i in range(1,11,1):
    decision_tree = tree.DecisionTreeClassifier(criterion = 'gini', max_depth = i, random_state = 0)
    X = music.drop('good',1)
    Y = music['good']
    decision_tree.fit(X,Y)

    dot_data = tree.export_graphviz(decision_tree, out_file = None, feature_names = X.columns, class_names=['Male', 'Female'], filled=True)
    graph = pydotplus.graph_from_dot_data(dot_data)
    Image(graph.create_png())
    print('Depth: {}'.format(i))
    print('Model Performance: {:.2%}.\n'.format(cross_val_score(decision_tree, X, Y, cv=10).mean()))

print("--- Runtime: %s seconds. ---" % (timeit.default_timer() - start_time))

Depth: 1
Model Performance: 82.87%.

Depth: 2
Model Performance: 82.87%.

Depth: 3
Model Performance: 82.87%.

Depth: 4
Model Performance: 82.79%.

Depth: 5
Model Performance: 82.61%.

Depth: 6
Model Performance: 82.23%.

Depth: 7
Model Performance: 81.87%.

Depth: 8
Model Performance: 81.55%.

Depth: 9
Model Performance: 80.92%.

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.96399 to fit

Depth: 10
Model Performance: 80.20%.

--- Runtime: 33.5328674 seconds. ---
